## 『本次練習內容』
#### 學習如何搭建Vgg16模型

## 『本次練習目的』
  #### 熟悉經典Vgg模型的搭建
  #### 時間較為充裕的學員可以嘗試搭建Vgg19當作練習

CNN 演進¶
LeNet CNN start
AlexNet cnn-cv
Vgg classic
Inception googlenet v1-v4
ResNet 奠定深層cnn structure

![vgg16_describe](VGG16.png)

### LeNet

Yann LeCun 於 1998年發表的 『LeNet』為CNN架構的開宗始祖，其正式帶入了卷積的概念。論文中採用 5*5的卷積核，Strides為 1，並運用MaxPooling強化特徵，輸出層採用了Radial Basis Function函式，即徑向歐式距離函式。由於LeNet歷史已經相當悠久。

### AlexNet
AlexNet為 2012年ImageNet的冠軍，以作者Alex命名，其出現大幅打破了過去的紀錄，將top-5 error降低至15.3% ，可說是正式將CNN帶入Computer Vision領域的最大功臣。

Structure 5 convolutions and 3 fc RELU raise relu augmentation dropout LRN local response normalization 這部分影響不大

### ReLU
ReLU的概念相當簡單，小於0的為0，大於0的值就是f(x)=x。ReLU相較於Sigmoid的好處為『比較不容易造成梯度消失』：Back Propagation 透過鏈鎖率來計算損失函數的梯度，由於Sigmoid的導函數最大值為0.25，當神經網路搭得比較深時，容易造成梯度消失 。

### Dropout
Dropout觀念相當直觀，就是在每次 Forward Propagation 時隨機關閉特定比例的神經元，避免模型 Overfitting。AlexNet 中使用比例為 0.5，就是隨機關閉一半的神經元，目前較為常見的比例介於 0.1 - 0.3。

### Augmentation
Data Augmentation 為當今訓練模型時常見的技巧，AlexNet原文中是隨機裁切224*244的區域，並做鏡射與放大。

### LRN
LRN 顧名思義也是一種 Normalization 的方式，不過在近期研究中，發現添加 LRN 結構在模型中並不會提高準度，反而會增加記憶體使用量，因此已經沒有什麼文獻在使用 LRN

### Vgg
Vgg算是相當經典的架構，不少CV框架都是使用Vgg為Backbone，Vgg其實有相當多種版本，然而Vgg-16、Vgg-19效果最好，因此以這兩種版本最為常見。Vgg 論文中提到，3*3 卷積核能在參數更少的狀態下，達到跟 5*5 或是更大卷積核一樣的效果，兩層 3*3 卷積核的 Receptive Field 與一層的5*5卷積核一樣，然而參數卻是 0.72 倍。

---

##### 請參考下面結構圖搭建Vgg16模型

![vgg16](VGG16_p.png)

In [31]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K



def VGG16(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
  
    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



In [32]:
model = VGG16(include_top=False)

In [33]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
def VGG19(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
  
    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



In [ ]:
model = VGG19(include_top=False)

In [ ]:
model.summary()